In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [9]:
def extract_surface_lines_np(brep_str):
    lines = brep_str.strip().split('\n')
    surface_stop = 0
    surface_line = 0
    # for i, line in enumerate(lines):
    #     if line.startswith('Surface'):
    #         num_surfaces = int(line[-1])
    #         surface_line = i
    for i, line in enumerate(lines):
        if line.startswith('Surface'):
            surface_line = i
        if line.startswith('Triangulations'):
            surface_stop = i
            break

    surfaces = []

    for i in range(surface_line + 1, surface_stop):
        surfaces.append(lines[i].strip().split(' '))

    surfaces_flat = []
    for i in surfaces:
        for j in i:
            surfaces_flat.append(float(j))

    # maxlen = 1024
    # if len(surfaces_flat) < maxlen:
    #     for i in range(0, maxlen - len(surfaces_flat)):
    #         surfaces_flat.append(float(0))
    return np.array(surfaces_flat, dtype=np.float32)

In [10]:
# test function
with open('models/cone/a cone with a 10.00mm diameter and a 20.00mm height.brep', 'r', encoding='utf-8') as f:
    b = f.read()
    print(extract_surface_lines_np(b))

[ 3.          0.          0.         20.          0.          0.
 -1.         -1.          0.         -0.          0.          1.
  0.          0.          0.24497867  1.          0.          0.
  0.          0.          0.         -1.         -1.          0.
 -0.          0.          1.          0.        ]


In [11]:
import os

shapes = ['cone', 'cube', 'cylinder', 'sphere']

file_list = [os.listdir('models/' + shape) for shape in shapes]

X = []
y = []

for shape in shapes:
    y.append([shape]*len(os.listdir('models/' + shape)))

# flatten
file_list = [item for items in file_list for item in items]
y = [item for items in y for item in items]

for i in range(0, len(file_list)):
    with open('models/' + y[i] + '/' + file_list[i], 'r', encoding='utf-8') as f:
        brep_str = f.read()
        X.append(extract_surface_lines_np(brep_str))

In [15]:
from numpy import float32
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

maxlen = 1024
X_pad = pad_sequences(X, maxlen=maxlen, dtype='float32', padding='post', truncating='post', value=0.0)
X_pad = X_pad[..., None]

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_arr = np.asarray(y_encoded, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(X_pad, y_arr, test_size=0.2, random_state=3)

model = models.Sequential([
    layers.Input(shape=(maxlen, 1)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [17]:
model_trained = model.fit(X_train, y_train, epochs=10, validation_split=0.1, batch_size=64)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0290 - val_accuracy: 1.0000 - val_loss: 0.0155
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0217 - val_accuracy: 1.0000 - val_loss: 0.0116
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0172 - val_accuracy: 1.0000 - val_loss: 0.0093
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0142 - val_accuracy: 1.0000 - val_loss: 0.0078
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0120 - val_accuracy: 1.0000 - val_loss: 0.0068
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0106 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0095 - val_accuracy: 1.0000 - val_loss: 0.0054
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0086 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 9

In [1]:
# n-dim tensor?
# standard scaler?
# dense or convolutional?

# add metrics
# compare methods